In [1]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as stp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
!pip install gradio

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install typing_extensions

In [5]:
import gradio as gr

In [6]:
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')
df_cp = df.copy()

In [7]:
df_names = df['name'].copy()

In [8]:
df.isnull().sum()

df.drop(['accomplishments_segment','education_segment','emails','misc_segment','name','objectives_segment','phone','projects_segment','skills_segment','text',
         'university_0','university_1','university_2','university_3','university_4','university_5','url','work_segment'],axis=1,inplace=True)
df.shape

print('After removing the columns from dataset..')
df.isnull().sum()

df['degree'].fillna(df.degree.mode()[0],inplace=True)
df['links'].fillna('Missing',inplace=True)
df['work_experience'].fillna(0,inplace=True)
df['job_titles'].fillna('No Job',inplace=True)

all_degrees = ''
for i in df.degree:
  if len(all_degrees) == 0:
    all_degrees = i
  else:
    all_degrees = all_degrees + ' , ' + i

all_degrees = all_degrees.split(',')
all_degrees = [re.sub("[\s.]","",i).upper() for i in all_degrees]
unique_degrees = set(all_degrees)
unique_degrees

df['bachelor_degrees'] = 'No Degree'
df['master_degrees'] = 'No Degree'
df['docterte_degrees'] = 'No Degree'
df['profiles'] = 'No Profile'

After removing the columns from dataset..


In [9]:
ind = 0
for i in df.degree:
  lst = re.sub("[\s.]","",i).upper( )
  # print(lst)
  for j in lst.split(','):
    if j in ['BE', 'BS', 'BSC', 'BTECH']:
      if df.loc[ind,'bachelor_degrees'] == 'No Degree':
        df.loc[ind,'bachelor_degrees'] = j
      else:
        df.loc[ind,'bachelor_degrees'] = df.loc[ind,'bachelor_degrees'] + ' , '+ j
    elif j in ['ME', 'MS', 'MSC', 'MTECH']:
      if df.loc[ind,'master_degrees'] == 'No Degree':
        df.loc[ind,'master_degrees'] = j
      else:
        df.loc[ind,'master_degrees'] = df.loc[ind,'master_degrees'] + ' , '+ j
    elif j == 'PHD':
      if df.loc[ind,'docterte_degrees'] == 'No Degree':
        df.loc[ind,'docterte_degrees'] = j
      else:
        df.loc[ind,'docterte_degrees'] = df.loc[ind,'docterte_degrees'] + ' , '+ j
  ind +=1

df


,degree,job_titles,links,skills,work_experience,bachelor_degrees,master_degrees,docterte_degrees,profiles
0,"MS , B.TECH , PhD","full stack developer , student","https://github.com/ramsuthar305/SIH2019 , http...","algorithms , interview , visualisation , robot...",0.0,BTECH,MS,PHD,No Profile
1,"B.E , MS , M.TECH",software engineer,pooj.vs,"core java , algorithms , sql tools , Jenifer J...",0.0,BE,"MS , MTECH",No Degree,No Profile
2,"MS , B.TECH , BS",No Job,https://www.linkedin.com/in/ashwin-khandelwal-...,"digital signal processing , algorithms , capac...",0.0,"BTECH , BS",MS,No Degree,No Profile
3,"B.E , MS , BS","work from home , support analyst , software an...",Missing,"big data analytics , award , training , comput...",1.0,"BE , BS",MS,No Degree,No Profile
4,"MS , B.TECH , BS","work from home , classifier",https://github.com/ajay1013/Titanic-ML-from-Di...,"functionality , algorithms , spark , data visu...",0.0,"BTECH , BS",MS,No Degree,No Profile
...,...,...,...,...,...,...,...,...,...
1568,"B.E , MS , M.TECH","branch manager , embedded software developer ,...","https://www.linkedin.com/in , https://grv20.gi...","digital communication , api , atom , robot , c...",2.0,BE,"MS , MTECH",No Degree,No Profile
1569,"MS , BS","devops , senior android developer , team lead","com.gokada , https://play.google.com/store/app...","reporting , functionality , socket.io , mockit...",2.0,BS,MS,No Degree,No Profile
1570,"B.S , M.Sc , M.S , BS , MS","devops engineer , traffic manager , release ma...",https://freesearch.naukri.com/preview/printResume,"version control , renewals , nfs , telugu , vi...",12.0,"BS , BS","MSC , MS , MS",No Degree,No Profile
1571,MS,classifier,Missing,"natural language , matplotlib , boost , custom...",0.0,No Degree,MS,No Degree,No Profile


In [10]:
ind = 0
for i in df.links:
  lst = re.sub("[\s]","",i)
  # print(lst)
  for j in lst.split(','):
    if j.find('github') != -1:
      df.loc[ind,'profiles'] = 'Github'
    elif j.find('linkedin') != -1:
      if df.loc[ind,'profiles'] == 'No Profile':
        df.loc[ind,'profiles'] = 'Linkedin'
      else:
        df.loc[ind,'profiles'] = df.loc[ind,'profiles'] + ' , ' + 'Linkedin'
  ind +=1


In [11]:
ind = 0
for i in df.work_experience:
  if i < 0:
    df.loc[ind,'work_experience'] = i*-1
  elif i > 40:
    df.loc[ind,'work_experience'] = 0
  ind +=1
df['work_experience'] = df['work_experience'].astype(int)

with open('/content/drive/MyDrive/Job Description.txt','r',encoding = 'utf-8') as f:
  file_desc_lst =  [r.replace('\n', '') for r in f.readlines()]

job_description = ''

for i in file_desc_lst:
  if len(job_description) == 0:
    job_description = str(i)
  else:
    job_description = job_description + ' ' + str(i)
print(job_description)

all_resume_text = []

for i in df.iloc[:].values:
  s = ''
  for j in list(i):
    if len(s) == 0:
      s = str(j)
    else:
      s = s + ' , ' + str(j)
  all_resume_text.append(s)


Job Area:Engineering Group, Engineering Group > Systems Engineering                    General Summary                    Bachelor's degree in Engineering, Information Systems, Computer Science, or related field.                   / 12+ years Software Engineering or related work experience.                   / 12+ years experience with Programming Language such as C, C++/ Python, etc/                /                    Desired Skills And Aptitudes                   / Strong theoretical understanding of model architectures of various object classification, object detection models, recommender systems, NLP, Text and voice processing models, 3D models                   / Experience in creating new Neural NW architectures for different use cases and the use of correct datasets, achieving the desired target accuracy                   / Forward looking attitude, analyse upcoming trends in neural NWs and technics                   / Strong understanding of various model quantization techniqu

In [12]:
lemmatizer = WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in analyzer(doc) if w not in set(stp.words('english')))

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


In [13]:
def get_tf_idf_cosine_similarity(job_desc, all_resumes):
    tf_idf_vect = TfidfVectorizer(analyzer=stemmed_words)
    job_description = [job_desc]
    tf_idf_desc_vector = np.array(tf_idf_vect.fit_transform(job_description).todense())
    tf_idf_desc_vector = tf_idf_desc_vector.reshape(1, -1)
    tf_idf_resume_vector = np.array(tf_idf_vect.transform(all_resumes).todense())
    cosine_similarity_list = []
    for resume_vector in tf_idf_resume_vector:
      cosine_similarity_list.append(cosine_similarity(tf_idf_desc_vector, resume_vector.reshape(1,-1))[0][0])
    return cosine_similarity_list


In [14]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

import numpy as np


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [15]:
cos_sim_list = get_tf_idf_cosine_similarity(job_description, all_resume_text)

In [16]:
zipped_resume_rating = zip(cos_sim_list,df_cp.name,[x for x in range(len(df))])
sorted_resume_rating_list = sorted(zipped_resume_rating, key = lambda x: x[0], reverse=True)

resume_score = [round(x*100,2) for x in cos_sim_list]
pd.concat([df_cp.name,pd.DataFrame(resume_score,columns=['resume_score(%)'])],axis=1).sort_values(by=['resume_score(%)'],ascending=False).head(10)


,name,resume_score(%)
464,Rucha Patwardhan,57.94
1095,Gary,51.66
1542,Sanjay,49.91
98,Gautham Patel,49.06
236,Jenifer Johnson,47.08
852,Ameer Sharma,46.55
1351,Sanjay,45.86
544,Jenifer Johnson,45.57
75,Vineet Kumar,45.43
1081,Udaya Saikiran,45.30


In [25]:
# Encoding categorical variables
label_encoder = LabelEncoder()
df['bachelor_degrees'] = label_encoder.fit_transform(df['bachelor_degrees'])
df['master_degrees'] = label_encoder.fit_transform(df['master_degrees'])
df['docterte_degrees'] = label_encoder.fit_transform(df['docterte_degrees'])
df['profiles'] = label_encoder.fit_transform(df['profiles'])

# Extracting features and target variable
X = df[['bachelor_degrees', 'master_degrees', 'docterte_degrees', 'profiles', 'work_experience']]
y = cos_sim_list  # Using cosine similarity as the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)


# Encoding categorical variables
label_encoder = LabelEncoder()
df['bachelor_degrees'] = label_encoder.fit_transform(df['bachelor_degrees'])
df['master_degrees'] = label_encoder.fit_transform(df['master_degrees'])
df['docterte_degrees'] = label_encoder.fit_transform(df['docterte_degrees'])
df['profiles'] = label_encoder.fit_transform(df['profiles'])

# Extracting features and target variable
X = df[['bachelor_degrees', 'master_degrees', 'docterte_degrees', 'profiles', 'work_experience']]
y = cos_sim_list  # Using cosine similarity as the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results_df.head(10))
resume_score = [round(x*100,2) for x in cos_sim_list]
pd.concat([df_cp.name,pd.DataFrame(resume_score,columns=['resume_score(%)'])],axis=1).sort_values(by=['resume_score(%)'],ascending=False).head(10)


from sklearn.metrics import r2_score

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Convert R-squared to percentage
accuracy_percentage = r2 * 100

print(f'R-squared: {r2}')
print(f'Accuracy: {accuracy_percentage:.2f}%')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 MB 8.2 MB/s eta 0:00:00
Mean Squared Error: 0.006205382361122717
     Actual  Predicted
0  0.157359   0.225835
1  0.223542   0.247486
2  0.441265   0.240520
3  0.307784   0.238258
4  0.450415   0.295566
5  0.127618   0.208486
6  0.267946   0.276785
7  0.265657   0.241988
8  0.308132   0.276785
9  0.433291   0.247348
R-squared: 0.031389112684515386
Accuracy: 3.14%


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [22]:
def get_resume_score(job_description, df, all_resume_text):
    cos_sim_list = get_tf_idf_cosine_similarity(job_description, all_resume_text)

    # Encode categorical variables
    label_encoder = LabelEncoder()
    df['bachelor_degrees'] = label_encoder.fit_transform(df['bachelor_degrees'])
    df['master_degrees'] = label_encoder.fit_transform(df['master_degrees'])
    df['docterte_degrees'] = label_encoder.fit_transform(df['docterte_degrees'])
    df['profiles'] = label_encoder.fit_transform(df['profiles'])

    # Extract features and target variable
    X = df[['bachelor_degrees', 'master_degrees', 'docterte_degrees', 'profiles', 'work_experience']]
    y = cos_sim_list

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    %Pip install scikit-learn-intelex
    from sklearnex import patch_sklearn
    patch_sklearn()
    # Initialize the Random Forest Regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = rf_model.predict(X_test)

    # Calculate Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    print(results_df.head(10))

    # Calculate R-squared
    r2 = r2_score(y_test, y_pred)
    accuracy_percentage = r2 * 100

    print(f'R-squared: {r2}')
    print(f'Accuracy: {accuracy_percentage:.2f}%')
    cos_sim_list=[x*100 for x in cos_sim_list]
    return pd.concat([df_names, pd.DataFrame(cos_sim_list, columns=['resume_score(%)'])], axis=1).sort_values(by=['resume_score(%)'], ascending=False).head(10)



# Gradio interface function
def gr_interface(job_description_file):
    #gr.Warning(job_description_file.name)
    with open(job_description_file.name, 'r', encoding='utf-8') as f:
        file_desc_lst =  [r.replace('\n', '') for r in f.readlines()]
    job_description = ''
    for i in file_desc_lst:
      if len(job_description) == 0:
        job_description = str(i)
      else:
        job_description = job_description + ' ' + str(i)
    all_resume_text = []
    for i in df.iloc[:].values:
      s = ''
      for j in list(i):
        if len(s) == 0:
          s = str(j)
        else:
          s = s + ' , ' + str(j)
      all_resume_text.append(s)
    top_resumes = get_resume_score(job_description, df, all_resume_text)
    table_html=top_resumes.reset_index().to_html(index=False)
    model_explanation = """
    <div class="model-explanation">
        <h3>Model Explanation</h3>
        <p>This model uses a Random Forest Regressor to score resumes based on their similarity to the provided job description.
        It considers various features such as education, profiles, and work experience to make predictions.</p>
    </div>
    """

    # Concatenate the model explanation and the table HTML
    full_html = model_explanation + table_html
    return full_html
iface = gr.Interface(
    fn=gr_interface,
    inputs=gr.File(label="Upload Job Description", type="filepath"),
    outputs=gr.HTML(label="Top 10 Resumes"),
    theme="huggingface",
    live=True,
    title="Resume Scorer",
    description="Upload a job description file and get the top 10 matching resumes.",
    css="""
    .gr-outer-container {
        max-width: 800px;
        margin: auto;
        background-color: #f4f4f4;  /* Set your desired background color */
        font-family: 'Arial', sans-serif;  /* Set your desired font */
    }
    .gr-interface {
        padding: 20px;
        border: 1px solid #ccc;
        border-radius: 10px;
        box-shadow: 0px 0px 10px 0px #ccc;  /* Add box shadow for a nice effect */
    }
    .gr-model {
        margin-bottom: 20px;
    }
    .gr-output {
        background-color: #fff;  /* Set the background color of the output area */
        padding: 10px;
        border-radius: 5px;
    }
    .gr-model-title {
        font-size: 20px;  /* Set the font size of the model title */
        color: #333;  /* Set the font color of the model title */
    }
    """,
)

# Launch the Gradio interface
iface.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:529: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-65b4a140-0b51fea86a33239d757d169c;f82d7565-b803-4958-8494-1849fdc0d09c)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://15efbcd8c46cc9d418.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://15efbcd8c46cc9d418.gradio.live
